In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
real_weather = pd.read_csv('data/jfk_weather_cleaned1.csv',usecols=[1])
fake_weather_noise = pd.read_csv('data1/generatedata.csv',usecols=[1])
real_weather.shape

In [3]:
data_csv=fake_weather_noise[:2000]
data_csv = data_csv.dropna()
dataset = data_csv.values
dataset = dataset.astype('float32')
max_value = np.max(dataset)
min_value = np.min(dataset)
scalar = max_value - min_value 
dataset = list(map(lambda x: x / scalar, dataset)) 

In [4]:
def create_dataset(dataset, look_back=2):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

# 创建好输入输出
data_X, data_Y = create_dataset(dataset)

In [5]:
train_size = int(len(data_X) * 0.7)
test_size = len(data_X) - train_size
train_X = data_X[:train_size]
train_Y = data_Y[:train_size]
test_X = data_X[train_size:]
test_Y = data_Y[train_size:]

In [6]:
import torch

train_X = train_X.reshape(-1, 1, 2)
train_Y = train_Y.reshape(-1, 1, 1)
test_X = test_X.reshape(-1, 1, 2)

train_x = torch.from_numpy(train_X)
train_y = torch.from_numpy(train_Y)
test_x = torch.from_numpy(test_X)

In [7]:
from torch import nn
from torch.autograd import Variable

class lstm(nn.Module):
    def __init__(self,input_size=2,hidden_size=4,output_size=1,num_layer=2):
        super(lstm,self).__init__()
        self.layer1 = nn.LSTM(input_size,hidden_size,num_layer)
        self.layer2 = nn.Linear(hidden_size,output_size)
    
    def forward(self,x):
        x,_ = self.layer1(x)
        s,b,h = x.size()
        x = x.view(s*b,h)
        x = self.layer2(x)
        x = x.view(s,b,-1)
        return x

model = lstm(2, 4,1,2)

In [8]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [9]:
# 开始训练
for e in range(500):
    var_x = Variable(train_x)
    var_y = Variable(train_y)
    # 前向传播
    out = model(var_x)
    loss = criterion(out, var_y)
    # 反向传播
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (e + 1) % 100 == 0: # 每 100 次输出结果
        print('Epoch: {}, Loss: {:.5f}'.format(e + 1, loss.item()))
        
torch.save(model, 'data1/fakedata.pkl')

Epoch: 100, Loss: 0.02434
Epoch: 200, Loss: 0.00600
Epoch: 300, Loss: 0.00444
Epoch: 400, Loss: 0.00363
Epoch: 500, Loss: 0.00324


In [ ]:
data_X = test_X.reshape(-1, 1, 2)
data_X = torch.from_numpy(data_X)
var_data = Variable(data_X)
save_path = 'data1/fakedata.pkl'
model=torch.load(save_path)

model = model.eval() 

pred_test = model(var_data) 
pred_test = pred_test.view(-1).data.numpy()

plt.plot(test_Y[200:400], label='real')
plt.plot(pred_test[200:400], label='G-prediction',ls=':')

plt.legend(loc='best')

from sklearn.metrics import mean_squared_error
test_score = mean_squared_error(pred_test[200:400], test_Y[200:400])
print('Test MSE: %.3f' % test_score)